# Analyze Data

In [ ]:
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(table1, warn.conflicts = FALSE)
library(broom, warn.conflicts = FALSE)
library(forcats, warn.conflicts = FALSE)
library(splines)
library(emmeans)


library(survival, warn.conflicts = FALSE)
library(survminer, warn.conflicts = FALSE)
library(stringr)

In [ ]:
library(truveta.research)

In [ ]:
source(here::here("wrangle_scripts", "R", "write_as_xtable.r"))
source(here::here("wrangle_scripts", "R", "survival_helpers.r"))
source(here::here("wrangle_scripts", "R", "analyze_time_indep.r"))
source(here::here("wrangle_scripts", "R", "analyze_time_dependent.r"))

In [ ]:
tracking_dir <- here::here("tracking")
datadefs_dir <- file.path(tracking_dir, "datadefs")
hashsum_dir <- file.path(tracking_dir, "hashsums")
dir.create(hashsum_dir, recursive = TRUE, showWarnings = FALSE)

results_dir <- here::here("results")
data_dir <- here::here("data")
dir.create(data_dir, recursive = TRUE, showWarnings = FALSE)
dir.create(results_dir, recursive = TRUE, showWarnings = FALSE)

In [ ]:
# constants

one_year <- as.numeric(lubridate::dyears(1))
one_day <- as.numeric(lubridate::ddays(1))
one_week <- as.numeric(lubridate::dweeks(1))

two_weeks <- as.numeric(lubridate::dweeks(2))

wash_out <- as.numeric(lubridate::ddays(28))

In [ ]:
initialize_theme_truveta(figsize = c(8, 8))

# starts with black
cbPalette <- c("#999999", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7")

In [ ]:
summary_analysis <- list()

In [ ]:
theme_truveta <- 
  function (figsize = NULL) {
    if (!is.null(figsize)) {
        options(repr.plot.width = figsize[1], repr.plot.height = figsize[2])
    }
    else {
        options(repr.plot.width = getOption("truveta.plot.width"), 
            repr.plot.height = getOption("truveta.plot.height"))
    }
    variant <- getOption("truveta.theme.variant")
    ggplot2::theme_minimal() + 
      ggplot2::theme(
        legend.position = "top", 
        legend.justification = "left", 
        legend.title = ggplot2::element_blank(), 
        text = ggplot2::element_text(family = 'Open Sans', size = 15), 
        plot.title = 
          ggplot2::element_text(
            size = 20, 
            colour = ifelse(variant == "official", official_primary_color2, "#5068DA"), hjust = 0), 
        #axis.title.x = ggplot2::element_text(hjust = 1),
        axis.text.y = element_text(hjust = 0.5, colour = 'black'),
        axis.text.x = element_text(colour = 'black')
      )
  }

## Load Data

In [ ]:
df <- read_parquet_table(file.path(data_dir, "feature_table.parquet"), hashsum_dir)

In [ ]:
colnames(df)

## Setup and utilities

There are 3 major data versions to look at.

- CDC long COVID definition as outcome
- icd10 + snomed long COVID definition as outcome
- remove patients with 0 time till outcome
    - currently included 

# main analyses

## Simple analysis

restrict to either "vaccinated before covid" or "never vaccinated"

ignore effect of boosters

### simple w/ CDC outcome (symptoms)

In [ ]:
simple_cdc <- simple_analysis(df, time_long_covid)

In [ ]:
summary_analysis$n_simple_cdc <- dplyr::n_distinct(simple_cdc$simple$person_id)


# km graph
g <-
  simple_cdc$simple_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till developing Long COVID symptoms', 52),
    subtitle = 'Vaccination status at time of COVID infection'
  )

write_ggplot(g, file.path(results_dir, "km_time_simple.png"))

g

# cox model summary table
write_table(
  simple_cdc$simple_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_simple.csv')
)


# cox model estimates
gh <- 
  simple_cdc$simple_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of developing Long COVID symptoms',
    subtitle = 'Vaccination status at time of COVID infection'
  )

write_ggplot(gh, file.path(results_dir,"cox_simple_hr.png"))


# hazard table
caption <- 
  paste0(
    "Estimated hazard ratios for risk of developing Long COVID symptoms based on vaccination status at time of COVID infection. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(simple_cdc$sp_table, "cox_simple_emm_pairs", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(simple_cdc$sp_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(
    mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)
  ) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of developing Long COVID symptoms', 47),
    subtitle = 'Vaccination status at time of COVID infection',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

write_ggplot(gr, file.path(results_dir, 'cox_simple_emm_pairs.png'))

gr

### simple w/ diagnosis outcome (icd10/snomed codes)

In [ ]:
# alternate diagnosis based outcome

simple_diagnosis <- simple_analysis(df, time_long_covid_diagnosis)

In [ ]:
summary_analysis$n_simple_diagnosis <- dplyr::n_distinct(simple_diagnosis$simple$person_id)


# km graph
g <- 
  simple_diagnosis$simple_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till being diagnosed with Long COVID', 51),
    subtitle = 'Vaccination status at time of COVID infection'
  )
    
write_ggplot(g, file.path(results_dir, "km_time_simple_alt.png"))

g


# cox model summary table
write_table(
  simple_diagnosis$simple_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_simple_alt.csv')
)


# cox model estimates
gh <- 
  simple_diagnosis$simple_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of being diagnosed with Long COVID',
    subtitle = 'Vaccination status at time of COVID infection'
  )

write_ggplot(gh, file.path(results_dir,"cox_simple_hr_alt.png"))


# hazard table
caption <- 
  paste0(
    "Estimated hazard ratios for risk of being diagnosed with Long COVID based on vaccination status at time of COVID infection. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(simple_diagnosis$sp_table, "cox_simple_alt_emm_pairs", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(simple_diagnosis$sp_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(
    mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)
  ) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of being diagnosed with Long COVID', 46),
    subtitle = 'Vaccination status at time of COVID infection',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

gr

write_ggplot(gr, file.path(results_dir, 'cox_simple_alt_emm_pairs.png'))

## Time dependent vaccination/boosting

### time dependent w/ CDC outcome (symptoms)

In [ ]:
timedep_cdc <- time_dependent_analysis(df, time_long_covid)

In [ ]:
summary_analysis$n_timedep_cdc <- dplyr::n_distinct(timedep_cdc$time$person_id)


# km graph
g <-
  timedep_cdc$time_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till developing Long COVID symptoms', 52),
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(g, file.path(results_dir,"km_time_dep.png"))  

g


# cox model summary table
write_table(
  timedep_cdc$time_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_dep.csv')
)


# cox model estimates
gh <- 
  timedep_cdc$time_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of developing Long COVID symptoms',
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(gh, file.path(results_dir, "cox_time_dep_hr.png"))


# hazard table
caption <- 
  paste0(
    "Estimated hazard ratios for risk of developing Long COVID symptoms associated with vaccination status where vaccination status is modeled as time-dependent covariates. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(timedep_cdc$time_table, "cox_time_dep_emm_pairs", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(timedep_cdc$time_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of developing Long COVID symptoms', 47),
    subtitle = 'Time-dependent vaccination status',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

gr

write_ggplot(gr, file.path(results_dir, 'cox_time_dep_emm_pairs.png'))

### time dependent w/ diagnosis outcome (icd10/snomed codes)

In [ ]:
timedep_diagnosis <- time_dependent_analysis(df, time_long_covid_diagnosis)

In [ ]:
summary_analysis$n_timedep_diagnosis <- dplyr::n_distinct(timedep_diagnosis$time$person_id)


# km graph
g <-
  timedep_diagnosis$time_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till being diagnosed with Long COVID', 51),
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(g, file.path(results_dir,"km_time_dep_alt.png"))  

g


# cox model summary table
write_table(
  timedep_diagnosis$time_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_dep_alt.csv')
)


# cox model estimates
gh <- 
  timedep_diagnosis$time_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of being diagnosed with Long COVID',
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(gh, file.path(results_dir,"cox_time_dep_hr_alt.png"))


# hazard table
caption <-
  paste0(
    "Estimated hazard ratios for risk of being diagnosed with Long COVID associated with vaccination status where vaccination status is modeled as time-dependent covariates. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(timedep_diagnosis$time_table, "cox_time_dep_alt_emm_pairs", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(timedep_diagnosis$time_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of being diagnosed with Long COVID', 46),
    subtitle = 'Time-dependent vaccination status',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

gr

write_ggplot(gr, file.path(results_dir, 'cox_time_dep_alt_emm_pairs.png'))

# various sensitivity analyses

## remove 0 time till outcome individuals

### time dependent w/ CDC outcome (symptoms), removing 0 time till outcome individuals

In [ ]:
timedep_nozero_cdc <- 
  time_dependent_analysis(df, time_long_covid, keep_zeroes = FALSE)

In [ ]:
summary_analysis$n_timedep_nozero_cdc <- 
  dplyr::n_distinct(timedep_nozero_cdc$time$person_id)


# km graph
g <-
  timedep_nozero_cdc$time_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till developing Long COVID symptoms', 52),
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(g, file.path(results_dir,"km_time_dep_nozero.png"))  

g


# cox model summary table
write_table(
  timedep_nozero_cdc$time_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_dep_nozero.csv')
)


# cox model estimates
gh <- 
  timedep_nozero_cdc$time_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of developing Long COVID symptoms',
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(gh, file.path(results_dir,"cox_time_dep_hr_nozero.png"))


# hazard table
caption <- 
  paste0(
    "Estimated hazard ratios for risk of developing Long COVID symptoms associated with vaccination status where vaccination status is modeled as time-dependent covariates. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(timedep_nozero_cdc$time_table, "cox_time_dep_nozero_emm_pairs", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(timedep_nozero_cdc$time_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of developing Long COVID symptoms', 47),
    subtitle = 'Time-dependent vaccination status',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

gr

write_ggplot(gr, file.path(results_dir, 'cox_time_dep_nozero_emm_pairs.png'))

### time dependent w/ diagnosis outcome (icd10/snomed codes), removing 0 time till outcome individausl

In [ ]:
timedep_nozero_diagnosis <- 
 time_dependent_analysis(df, time_long_covid_diagnosis, keep_zeroes = FALSE)

In [ ]:
summary_analysis$n_timedep_nozero_diagnosis <- 
  dplyr::n_distinct(timedep_nozero_diagnosis$time$person_id)


# km graph
g <-
  timedep_nozero_diagnosis$time_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till being diagnosed with Long COVID', 51),
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(g, file.path(results_dir,"km_time_dep_nozero_alt.png"))  

g


# cox model summary table
write_table(
  timedep_nozero_diagnosis$time_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_dep_nozero_alt.csv')
)


# cox model estimates
gh <- 
  timedep_nozero_diagnosis$time_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of being diagnosed with Long COVID',
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(gh, file.path(results_dir,"cox_time_dep_hr_nozero_alt.png"))


# hazard table
caption <-
  paste0(
    "Estimated hazard ratios for risk of being diagnosed with Long COVID associated with vaccination status where vaccination status is modeled as time-dependent covariates. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(timedep_nozero_diagnosis$time_table, "cox_time_dep_nozero_alt_emm_pairs", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(timedep_nozero_diagnosis$time_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of being diagnosed with Long COVID', 46),
    subtitle = 'Time-dependent vaccination status',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

gr

write_ggplot(gr, file.path(results_dir, 'cox_time_dep_nozero_alt_emm_pairs.png'))

## just vaccinations after

exclude everyone who got vaccinated before getting covid

this mimics some analyses i've seen that look only at populations vaccinated after covid infection.

we will still treat vaccination and boosting as time dependent covariates

### symptoms, time dependent, after

In [ ]:
timedep_after_cdc <- time_dependent_analysis(df, time_long_covid, only_after = TRUE)

In [ ]:
summary_analysis$n_timedep_after_cdc <- dplyr::n_distinct(timedep_after_cdc$time$person_id)


# km graph
g <-
  timedep_after_cdc$time_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till developing Long COVID symptoms', 52),
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(g, file.path(results_dir,"km_time_dep_after_cdc.png"))  

g


# cox model summary table
write_table(
  timedep_after_cdc$time_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_dep_after_cdc.csv')
)


# cox model estimates
gh <- 
  timedep_after_cdc$time_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of developing Long COVID symptoms',
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(gh, file.path(results_dir,"cox_time_dep_hr_after_cdc.png"))


# hazard table
caption <- 
  paste0(
    "Estimated hazard ratios for risk of developing Long COVID symptoms associated with vaccination status where vaccination status is modeled as time-dependent covariates. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(timedep_after_cdc$time_table, "cox_time_dep_after_emm_pairs", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(timedep_after_cdc$time_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of developing Long COVID symptoms', 47),
    subtitle = 'Time-dependent vaccination status',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

gr

write_ggplot(gr, file.path(results_dir, 'cox_time_dep_after_emm_pairs_cdc.png'))

### diagnosis, time dependent, after

In [ ]:
timedep_after_diagnosis <- time_dependent_analysis(df, time_long_covid_diagnosis, only_after = TRUE)

In [ ]:
summary_analysis$n_timedep_after_diagnosis <- dplyr::n_distinct(timedep_after_diagnosis$time$person_id)


# km graph
g <-
  timedep_after_diagnosis$time_km$km_graph +
  ggplot2::labs(
    title = stringr::str_wrap('Survival curves for time till being diagnosed with Long COVID', 51),
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(g, file.path(results_dir,"km_time_dep_after_diag.png"))  

g


# cox model summary table
write_table(
  timedep_after_diagnosis$time_cox$cox_table, 
  filepath = file.path(results_dir, 'cox_summary_dep_after_diag.csv')
)


# cox model estimates
gh <- 
  timedep_after_diagnosis$time_cox$cox_hr +
  ggplot2::labs(
    title = 'Hazard ratio for risk of being diagnosed with Long COVID',
    subtitle = 'Time-dependent vaccination status'
  )

write_ggplot(gh, file.path(results_dir,"cox_time_dep_hr_after_diag.png"))


# hazard table
caption <-
  paste0(
    "Estimated hazard ratios for risk of being diagnosed with Long COVID associated with vaccination status where vaccination status is modeled as time-dependent covariates. ",
    "Hazard ratios are presented with 95\\% confidence intervals."
  )
write_hazard_table(timedep_after_diagnosis$time_table, "cox_time_dep_after_emm_pairs_diag", results_dir, caption)


# hazard ratios
gr <- 
  ggplot2::ggplot(timedep_after_diagnosis$time_table, ggplot2::aes(x = ratio, y = comparison)) +
  ggplot2::geom_vline(xintercept = 1, linetype = 'dashed') +
  ggplot2::geom_pointrange(mapping = ggplot2::aes(xmin = conf.low, xmax = conf.high)) +
  theme_truveta() +
  scale_y_discrete(labels = function(x) str_wrap(x, width = 10)) +
  ggplot2::labs(
    title = stringr::str_wrap('Hazard ratio for risk of being diagnosed with Long COVID', 46),
    subtitle = 'Time-dependent vaccination status',
    x = 'Hazard Ratio',
    y = 'Comparison'
  )

gr

write_ggplot(gr, file.path(results_dir, 'cox_time_dep_after_emm_pairs_diag.png'))

# export summary stats (mostly population sizes)

In [ ]:
save(summary_analysis, file = file.path(results_dir, 'summary_analysis.rdata'))